In [66]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import sys

%matplotlib inline

# implementation based on http://songhuiming.github.io/pages/2017/08/20/build-recurrent-neural-network-from-scratch/

# deterministic random

In [8]:
# utils
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

In [41]:
class RNN():
    def __init__(self, word_dim, hidden_dim = 100, bptt_truncate = 4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.__init_weights()
        
    def __init_weights(self):
        self.U = np.random.uniform(-np.sqrt(1./self.word_dim), 
                                   np.sqrt(1./self.word_dim), 
                                   (self.hidden_dim, self.word_dim))
        self.V = np.random.uniform(-np.sqrt(1./self.hidden_dim), 
                                   np.sqrt(1./self.hidden_dim), 
                                   (self.word_dim, self.hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./self.hidden_dim), 
                                   np.sqrt(1./self.hidden_dim), 
                                   (self.hidden_dim, self.hidden_dim))
    
    def forward_propagation(self, x):
        """
        @param x: vector of inputs
        """
        T = len(x)
        s = np.zeros((T+1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        o = np.zeros((T, self.word_dim))
        for t in np.arange(T):
            s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]

    def predict(self, x):
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis = 1)

    def calculate_total_loss(self, x, y):
        L = 0
        # for each sentence ...
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            # we only care about our prediction of the "correct" words
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            # add to the loss based on how off we were
            L += -1 * np.sum(np.log(correct_word_predictions))
        return L

    def calculate_loss(self, x, y):
        # divide the total loss by the number of training examples
        N = np.sum((len(y_i) for y_i in y))
        return self.calculate_total_loss(x, y)/N   
    
    def sgd_step(self, x, y, learning_rate):
        dLdU, dLdV, dLdW = self.bptt(x, y)
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW
    
    def bptt(self, x, y):
        T = len(y)
        # perform forward propagation
        o, s = self.forward_propagation(x)
        # we will accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1   # it is y_hat - y
        # for each output backwards ...
        for t in np.arange(T):
            dLdV += np.outer(delta_o[t], s[t].T)    # at time step t, shape is word_dim * hidden_dim
            # initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # backpropagation through time (for at most self.bptt_truncate steps)
            # given time step t, go back from time step t, to t-1, t-2, ...
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print("Backprogation step t=%d bptt step=%d" %(t, bptt_step))
                dLdW += np.outer(delta_t, s[bptt_step - 1])
                dLdU[:, x[bptt_step]] += delta_t
                # update delta for next step
                dleta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1]**2)
        return [dLdU, dLdV, dLdW]

    def gradient_check(self, x, y, h = 0.001, error_threshold = 0.01):
        # calculate the gradient using backpropagation
        bptt_gradients = self.bptt(x, y)
        # list of all params we want to check
        model_parameters = ["U", "V", "W"]
        # gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            # get the actual parameter value from model, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print("performing gradient check for parameter %s with size %d. " %(pname, np.prod(parameter.shape)))
            # iterate over each element of the parameter matrix, e.g. (0,0), (0,1)...
            it = np.nditer(parameter, flags = ['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # save the original value so we can reset it later
                original_value = parameter[ix]
                # estimate the gradient using (f(x+h) - f(x-h))/2h
                parameter[ix] = original_value + h
                gradplus = self.calculate_total_loss([x], [y])
                parameter[ix] = original_value - h
                gradminus = self.calculate_total_loss([x], [y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # reset parameter to the original value
                parameter[ix] = original_value
                # the gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate the relative error (|x - y|)/(|x|+|y|)
                relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # if the error is too large fail the gradient check
                if relative_error < error_threshold:
                    print("Gradient check error: parameter = %s ix = %s" %(pname, ix))
                    print("+h Loss: %f" % gradplus)
                    print("-h Loss: %f" % gradminus)
                    print("Estimated gradient: %f" % estimated_gradient)
                    print("Backpropagation gradient: %f" % backprop_gradient)
                    print("Relative error: %f" % relative_error)
                    return
                it.iternext()
            print("Gradient check for parameter %s passed. " %(pname))

In [24]:
def train_with_sgd(model, X_train, y_train, learning_rate = 0.005, nepoch = 100, evaluate_loss_after = 5):
    # keep track of the losses so that we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: loss after num_examples_seen=%d epoch=%d: %f" %(time, num_examples_seen, epoch, loss))
            # adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5
                print("setting learning rate to %f" %(learning_rate))
            sys.stdout.flush()
        # for each training example...
        for i in range(len(y_train)):
            # one sgd step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [81]:
X_train = [
    [1,2,3],
    [2,3,4],
    [3,4,5]
]

y_train = [
    [4],
    [5],
    [6]
]

rnn = RNN(7, hidden_dim=5)
train_with_sgd(rnn, X_train, y_train)

2019-06-02 21:24:55: loss after num_examples_seen=0 epoch=0: 1.895406
2019-06-02 21:24:55: loss after num_examples_seen=15 epoch=5: 1.884926
2019-06-02 21:24:55: loss after num_examples_seen=30 epoch=10: 1.874388
2019-06-02 21:24:55: loss after num_examples_seen=45 epoch=15: 1.863778
2019-06-02 21:24:55: loss after num_examples_seen=60 epoch=20: 1.853084
2019-06-02 21:24:55: loss after num_examples_seen=75 epoch=25: 1.842292
2019-06-02 21:24:55: loss after num_examples_seen=90 epoch=30: 1.831392
2019-06-02 21:24:55: loss after num_examples_seen=105 epoch=35: 1.820373
2019-06-02 21:24:55: loss after num_examples_seen=120 epoch=40: 1.809222
2019-06-02 21:24:55: loss after num_examples_seen=135 epoch=45: 1.797930
2019-06-02 21:24:55: loss after num_examples_seen=150 epoch=50: 1.786487
2019-06-02 21:24:55: loss after num_examples_seen=165 epoch=55: 1.774884
2019-06-02 21:24:55: loss after num_examples_seen=180 epoch=60: 1.763112
2019-06-02 21:24:55: loss after num_examples_seen=195 epoch=6

/Users/mk/repos/smada/ci-rnn/venv/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [107]:
rnn.predict([1,2,3,4,5])

array([4, 2, 6, 3, 0])